In [2]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
## Read data from file
gen_table = pd.read_csv('gen_table_mixed.csv')
gen_table

,Amino Acid,Fitness
0,LQKCHRESPLQRSAAYLLVILLALAMFAAAALVLVCVIASTSTSSS...,42
1,QISDKPGTEEATLPHLLVMIILLAMFAAAAAALVAVCIILVASTST...,43
2,VRLGRRSLLNCQIPPYLLVILLLAFLAMFAAAALVLAVCVMVASTS...,42
3,CWCECRPPTTHTRFILLVIILLAFAAAALVPLVAVRCIILVATTTT...,40
4,SSKLSHLLRNAPSNAYLLVILLAMFAIALVLAAVTAVVMILVSTTQ...,42
5,RLCVRLFLPNKCTHSYLLVWILLAMFAIALLAAVAVVMILVSTTQT...,28
6,SSKLSHLLRNAPSNAYLLVILLAMFAIALVLAAVTAVVMILVSTTQ...,42
7,GSRCYRYCEPAGEVVLLVIILLAFAAAALVPLVAVCIILVATTTTP...,39
8,PPRTTYQASLSQSNYLLVILLAMFAAAALVFPAVAVVMILVVATTT...,48
9,GPSRRLGHVIWPTFPYLLVILLLALAMFAAAPALVLAVCVMVASTT...,40


In [4]:
gen_array = gen_table["Amino Acid"]
gen_fit = gen_table["Fitness"]
gen_array

0    LQKCHRESPLQRSAAYLLVILLALAMFAAAALVLVCVIASTSTSSS...
1    QISDKPGTEEATLPHLLVMIILLAMFAAAAAALVAVCIILVASTST...
2    VRLGRRSLLNCQIPPYLLVILLLAFLAMFAAAALVLAVCVMVASTS...
3    CWCECRPPTTHTRFILLVIILLAFAAAALVPLVAVRCIILVATTTT...
4    SSKLSHLLRNAPSNAYLLVILLAMFAIALVLAAVTAVVMILVSTTQ...
5    RLCVRLFLPNKCTHSYLLVWILLAMFAIALLAAVAVVMILVSTTQT...
6    SSKLSHLLRNAPSNAYLLVILLAMFAIALVLAAVTAVVMILVSTTQ...
7    GSRCYRYCEPAGEVVLLVIILLAFAAAALVPLVAVCIILVATTTTP...
8    PPRTTYQASLSQSNYLLVILLAMFAAAALVFPAVAVVMILVVATTT...
9    GPSRRLGHVIWPTFPYLLVILLLALAMFAAAPALVLAVCVMVASTT...
Name: Amino Acid, dtype: object

In [8]:
amino_acids = {
    'A': 0, 'R': 1, 'N': 2, 'D': 3, 'C': 4, 'Q': 5, 'E': 6, 'G': 7, 'H': 8, 'I': 9,
    'L': 10, 'K': 11, 'M': 12, 'F': 13, 'P': 14, 'S': 15, 'T': 16, 'W': 17, 'Y': 18, 'V': 19
}

blosum50_matrix = [
    [ 5, -2, -1, -2, -1, -1, -1,  0, -2, -1, -2, -1, -1, -3, -1,  1,  0, -3, -2,  0],
    [-2,  7, -1, -2, -4,  1,  0, -3,  0, -4, -3,  3, -2, -3, -3, -1, -1, -3, -1, -3],
    [-1, -1,  7,  2, -2,  0,  0,  0,  1, -3, -4,  0, -2, -4, -2,  1,  0, -4, -2, -3],
    [-2, -2,  2,  8, -4,  0,  2, -1, -1, -4, -4, -1, -4, -5, -1,  0, -1, -5, -3, -4],
    [-1, -4, -2, -4, 13, -3, -3, -3, -3, -2, -2, -3, -2, -2, -4, -1, -1, -5, -3, -1],
    [-1,  1,  0,  0, -3,  7,  2, -2,  1, -3, -2,  2,  0, -4, -1,  0, -1, -1, -1, -3],
    [-1,  0,  0,  2, -3,  2,  6, -3,  0, -4, -3,  1, -2, -3, -1,  -1, -1, -3, -2, -3],
    [ 0, -3,  0, -1, -3, -2, -3,  8, -2, -4, -4, -2, -3, -4, -2,  0, -2, -3, -3, -4],
    [-2,  0,  1, -1, -3,  1,  0, -2, 10, -4, -3,  0, -1, -1, -2, -1, -2, -3,  2, -4],
    [-1, -4, -3, -4, -2, -3, -4, -4, -4,  5,  2, -3,  2,  0, -3, -3, -1, -3, -1,  4],
    [-2, -3, -4, -4, -2, -2, -3, -4, -3,  2,  5, -3,  3,  1, -4, -3, -1, -2, -1,  1],
    [-1,  3,  0, -1, -3,  2,  1, -2,  0, -3, -3,  6, -2, -4, -1,  0, -1, -3, -2, -3],
    [-1, -2, -2, -4, -2,  0, -2, -3, -1,  2,  3, -2,  7,  0, -3, -2, -1, -1,  0,  1],
    [-3, -3, -4, -5, -2, -4, -3, -4, -1,  0,  1, -4,  0,  8, -4, -3, -2,  1,  4, -1],
    [-1, -3, -2, -1, -4, -1, -1, -2, -2, -3, -4, -1, -3, -4, 10, -1, -1, -4, -3, -3],
    [ 1, -1,  1,  0, -1,  0,  -1,  0, -1, -3, -3,  0, -2, -3, -1,  5,  2, -4, -2, -2],
    [ 0, -1,  0, -1, -1, -1, -1, -2, -2, -1, -1, -1, -1, -2, -1,  2,  5, -3, -2,  0],
    [-3, -3, -4, -5, -5, -1, -3, -3, -3, -3, -2, -3, -1,  1, -4, -4, -3, 15,  2, -3],
    [-2, -1, -2, -3, -3, -1, -2, -3,  2, -1, -1, -2, 0,  4, -3, -2, -2,  2,  8, -1],
    [ 0, -3, -3, -4, -1, -3, -3, -4, -4,  4,  1, -3,  1, -1, -3, -2,  0, -3, -1,  5]
]

In [9]:
class align:
    
    def __init__(self, str1, str2):
        self.str1 = str1
        self.str2 = str2
        self.gapP = -8
        self.score = [[0 for x in range(0,len(self.str1)+1)] for y in range(0,len(self.str2)+1)]
        self.direction = [[0 for x in range(0,len(self.str1)+1)] for y in range(0,len(self.str2)+1)]

    def my_print(self,array):
        for r in array:
            for c in r:
                print(f"{c:4}",end = ' ')
            print()

    def fill_matrix(self):
        for c in range(0,len(self.str1)+1):
            self.score[0][c] = c*self.gapP
            self.direction[0][c] = '-'
        for r in range(0,len(self.str2)+1):
            self.score[r][0] = r*self.gapP
            self.direction[r][0] = '|'
        self.direction[0][0] = 'X'
        # self.my_print(self.score)
        # self.my_print(self.direction)


    def NW(self):
        for c in range(1,len(self.str1)+1):
            for r in range(1,len(self.str2)+1):
                scoreD = self.score[r-1][c-1] + blosum50_matrix[amino_acids[self.str2[r-1]]][amino_acids[self.str1[c-1]]]
                scoreV = self.score[r-1][c] + self.gapP
                scoreH = self.score[r][c-1] + self.gapP
                self.score[r][c] = max(scoreD,scoreV,scoreH)
                if(self.score[r][c] == scoreD):
                    self.direction[r][c] = "\\"
                if(self.score[r][c] == scoreH):
                    self.direction[r][c] = "-"
                if(self.score[r][c] == scoreV):
                    self.direction[r][c] = "|"
        
    #     self.my_print(self.score)
    #     self.my_print(self.direction)


    def global_alignment(self):
        align1 = ""
        align2 = ""
        row = len(self.str2)
        col = len(self.str1)
        while(row > 0 or col > 0):
            if(self.direction[row][col] == "\\"):
                align1 += self.str1[col-1]
                align2 += self.str2[row-1]
                row -= 1
                col -= 1
            if(self.direction[row][col] == "-"):
                align1 += self.str1[col-1]
                align2 += "-"
                col -= 1
            if(self.direction[row][col] == "|"):
                align1 += "-"
                align2 += self.str2[row-1]
                row -= 1
        align1 = align1[::-1]
        align2 = align2[::-1]
        target = self.score[len(self.str2)][len(self.str1)]
        return align1,align2, target

In [10]:
def my_print(array):
        for r in array:
            for c in r:
                print(f"{c:4}",end = ' ')
            print()

In [11]:
gen_array

0    LQKCHRESPLQRSAAYLLVILLALAMFAAAALVLVCVIASTSTSSS...
1    QISDKPGTEEATLPHLLVMIILLAMFAAAAAALVAVCIILVASTST...
2    VRLGRRSLLNCQIPPYLLVILLLAFLAMFAAAALVLAVCVMVASTS...
3    CWCECRPPTTHTRFILLVIILLAFAAAALVPLVAVRCIILVATTTT...
4    SSKLSHLLRNAPSNAYLLVILLAMFAIALVLAAVTAVVMILVSTTQ...
5    RLCVRLFLPNKCTHSYLLVWILLAMFAIALLAAVAVVMILVSTTQT...
6    SSKLSHLLRNAPSNAYLLVILLAMFAIALVLAAVTAVVMILVSTTQ...
7    GSRCYRYCEPAGEVVLLVIILLAFAAAALVPLVAVCIILVATTTTP...
8    PPRTTYQASLSQSNYLLVILLAMFAAAALVFPAVAVVMILVVATTT...
9    GPSRRLGHVIWPTFPYLLVILLLALAMFAAAPALVLAVCVMVASTT...
Name: Amino Acid, dtype: object

In [12]:
pairwise_score = {}

score_matrix = [[0 for x in range(0, len(gen_array))] for y in range(0, len(gen_array))]

for i in range(0, len(gen_array)):
    for j in range(i+1, len(gen_array)):
        local = align(gen_array[i],gen_array[j])
        local.fill_matrix()
        local.NW()
        str1, str2, target = local.global_alignment()
        pairwise_score[(i+1,j)] = target
        score_matrix[i][j] = score_matrix[j][i] = target
        # score_matrix[j][i] = target


In [13]:
my_print(score_matrix)

   0  147  174  109   86   64   86  110  109  136 
 147    0  149  137   90   60   90  122  130  162 
 174  149    0   88   94   59   94  121  118  183 
 109  137   88    0   80   72   80  179  125  110 
  86   90   94   80    0  144  316  107  147  123 
  64   60   59   72  144    0  144   67   69   62 
  86   90   94   80  316  144    0  107  147  123 
 110  122  121  179  107   67  107    0  135  111 
 109  130  118  125  147   69  147  135    0  108 
 136  162  183  110  123   62  123  111  108    0 


In [14]:
## Adds al the rows together into a list
total_distances = np.sum(score_matrix, axis=1)

# Get ranking (sorted from highest to lowest distance)
ranking = np.argsort(total_distances)  

# Display ranked order
print("Evolutionary Age (Oldest to Youngest):")
for rank, idx in enumerate(ranking):
    print(f"Sequence {idx} (Total Distance: {total_distances[idx]})\n{gen_array[idx]}")


Evolutionary Age (Oldest to Youngest):
Sequence 5 (Total Distance: 741)
RLCVRLFLPNKCTHSYLLVWILLAMFAIALLAAVAVVMILVSTTQTTSRRR
Sequence 3 (Total Distance: 980)
CWCECRPPTTHTRFILLVIILLAFAAAALVPLVAVRCIILVATTTTSSRRRRKRR
Sequence 0 (Total Distance: 1021)
LQKCHRESPLQRSAAYLLVILLALAMFAAAALVLVCVIASTSTSSSRRRRKRRRR
Sequence 7 (Total Distance: 1059)
GSRCYRYCEPAGEVVLLVIILLAFAAAALVPLVAVCIILVATTTTPSTSRSRKRR
Sequence 2 (Total Distance: 1080)
VRLGRRSLLNCQIPPYLLVILLLAFLAMFAAAALVLAVCVMVASTSTSSTSRSRKRRRR
Sequence 1 (Total Distance: 1087)
QISDKPGTEEATLPHLLVMIILLAMFAAAAAALVAVCIILVASTSTSSSRRRKRRRR
Sequence 8 (Total Distance: 1088)
PPRTTYQASLSQSNYLLVILLAMFAAAALVFPAVAVVMILVVATTTSTSRSRKRRRR
Sequence 9 (Total Distance: 1118)
GPSRRLGHVIWPTFPYLLVILLLALAMFAAAPALVLAVCVMVASTTQTTSRRRRRRR
Sequence 6 (Total Distance: 1187)
SSKLSHLLRNAPSNAYLLVILLAMFAIALVLAAVTAVVMILVSTTQTTSRRRRRR
Sequence 4 (Total Distance: 1187)
SSKLSHLLRNAPSNAYLLVILLAMFAIALVLAAVTAVVMILVSTTQTTSRRRRRR
